# Cycle 3 UAT: Active Learning with Deformation

This notebook serves as the User Acceptance Test (UAT) for Cycle 3.
It verifies the "Safety Valve" logic: The system should detect rising uncertainty during a deformation simulation and interrupt the process before the structure degrades.

In [ ]:
import sys
import os
import shutil
from pathlib import Path
from unittest.mock import MagicMock, patch
import numpy as np
import matplotlib.pyplot as plt
from ase import Atoms
from ase.build import bulk
from omegaconf import OmegaConf

# 1. Setup Sys Path to include src
src_path = Path("../../src").resolve()
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

# 2. Check and Mock lammps module BEFORE importing anything from src
# This ensures we can run logic tests even if lammps python package is missing
HAS_LAMMPS_PKG = False
try:
    import lammps
    HAS_LAMMPS_PKG = True
    print("✅ LAMMPS Python package found.")
except ImportError:
    print("⚠️  LAMMPS Python package NOT found. Mocking it for import compatibility.")
    sys.modules["lammps"] = MagicMock()

# Imports from src
from core.orchestrator import ActiveLearningOrchestrator
from core.interfaces import AbstractPotential, AbstractOracle
from core.exceptions import UncertaintyInterrupt, OracleComputationError
from engines.lammps_mace import LammpsMaceDriver 
from oracles.qe_oracle import QeOracle

In [ ]:
# --- CHECK ENVIRONMENT ---
has_pw = shutil.which("pw.x") is not None
has_lammps_bin = shutil.which("lammps") is not None # or 'lmp'

print(f"pw.x found: {has_pw}")
print(f"lammps binary found: {has_lammps_bin}")

SIMULATION_MODE = False
if not has_pw or not has_lammps_bin or not HAS_LAMMPS_PKG:
    print("\n⚠️  MISSING BINARIES OR PACKAGE. Running in SIMULATION MODE.")
    print("    This checks the logic using Mocks for Driver and Oracle.")
    SIMULATION_MODE = True
else:
    print("\n✅ Environment OK. Running with Real Engines (if config matches).")

In [ ]:
# --- MOCKS & HELPERS ---

TRAJECTORY_LOG = []

class MockMacePotential(AbstractPotential):
    def __init__(self):
        self.call_count = 0
    
    def predict(self, atoms):
        # Return dummy energy/forces
        return 0.0, np.zeros((len(atoms), 3)), np.zeros((3, 3))

    def get_uncertainty(self, atoms):
        self.call_count += 1
        # Heuristic: linear growth with calls to simulate drifting into unknown territory
        base_u = 0.005 * self.call_count 
        return np.ones(len(atoms)) * base_u

    def train(self, training_data, **kwargs):
        print("MockPotential: Training triggered.")

    def save(self, path):
        print(f"MockPotential: Saved to {path}")
        
    def load(self, path):
        pass

class MockLammpsDriver:
    def __init__(self, potential, threshold=2.0):
        self.potential = potential
        self.threshold = threshold

    def run_md(self, atoms, script, steps, threshold=None):
        if threshold is not None:
            self.threshold = threshold
            
        print(f"MockDriver: Running MD with script:\n{script}")
        current_atoms = atoms.copy()
        status = "FINISHED"
        
        # Reset global log
        global TRAJECTORY_LOG
        TRAJECTORY_LOG.clear()

        # Parse stretch rate from script (approximate)
        erate = 0.01
        
        try:
            for i in range(steps):
                # Deform
                cell = current_atoms.get_cell()
                cell[0, 0] *= (1 + erate) 
                current_atoms.set_cell(cell, scale_atoms=True)
                
                # Check uncertainty
                u = self.potential.get_uncertainty(current_atoms)
                max_u = np.max(u)
                
                TRAJECTORY_LOG.append({"step": i, "max_u": max_u})
                print(f"Step {i}: Max U = {max_u:.4f} (Threshold: {self.threshold})")
                
                if max_u > self.threshold:
                    print("MockDriver: Uncertainty Threshold Exceeded!")
                    raise UncertaintyInterrupt(current_atoms, u)
                    
        except UncertaintyInterrupt as e:
            return e.atoms, "UNCERTAIN"
        
        return current_atoms, status

class MockOracle(AbstractOracle):
    def compute(self, atoms):
        print("MockOracle: Computing DFT...")
        res = atoms.copy()
        from ase.calculators.singlepoint import SinglePointCalculator
        res.calc = SinglePointCalculator(res, energy=-100.0, forces=np.zeros((len(atoms),3)))
        return res

class SiGenerator:
    def generate_initial_pool(self, n=5):
        print(f"SiGenerator: Creating {n} supercells.")
        atoms = bulk("Si", cubic=True)
        return [atoms * (2, 2, 2) for _ in range(n)]

In [ ]:
# --- SCENARIO CONFIGURATION ---

conf = OmegaConf.create({
    "experiment": {
        "work_dir": "uat_output",
        "max_cycles": 1,
        "box_size": 5.0
    },
    "exploration": {
        "md_steps": 30,
        "temperature": 300,
        "lammps_script": """
fix 1 all nvt temp 300 300 0.1
fix 2 all deform 1 x erate 0.01 remap x
""",
        "threshold": 0.1 # Threshold to trigger interrupt
    },
    "active_learning": {
        "threshold_ratio": 0.1
    },
    "training": {
        "energy_weight": 1.0,
        "forces_weight": 10.0
    }
})

In [ ]:
# --- EXECUTION ---

potential = MockMacePotential()
oracle = MockOracle()
generator = SiGenerator()

orchestrator = ActiveLearningOrchestrator(conf, potential, oracle, generator)

if SIMULATION_MODE:
    print("🚀 Starting UAT in SIMULATION MODE...")
    # We patch the LammpsMaceDriver inside core.orchestrator to use our MockDriver
    with patch("core.orchestrator.LammpsMaceDriver", side_effect=MockLammpsDriver):
        orchestrator.run_loop()
else:
    print("🚀 Starting UAT with REAL ENGINES...")
    # Even in real mode, we might want to use MockMacePotential if no model file exists,
    # but we assume the environment is set up if we are here.
    # Note: If no model file is loaded in Potential, real LammpsMaceDriver will fail.
    # For this UAT notebook, we prioritize the Logic Check.
    # Uncomment below to force MockDriver even if binaries exist (safer for Demo)
    # with patch("core.orchestrator.LammpsMaceDriver", side_effect=MockLammpsDriver):
    #    orchestrator.run_loop()
    orchestrator.run_loop()

In [ ]:
# --- VISUALIZATION ---

if 'TRAJECTORY_LOG' in globals() and TRAJECTORY_LOG:
    steps = [d['step'] for d in TRAJECTORY_LOG]
    uncerts = [d['max_u'] for d in TRAJECTORY_LOG]
    
    plt.figure(figsize=(10, 6))
    plt.plot(steps, uncerts, marker='o', linestyle='-', color='r', label='Max Uncertainty')
    plt.axhline(y=conf.exploration.threshold, color='k', linestyle='--', label='Threshold')
    plt.xlabel('MD Step')
    plt.ylabel('Uncertainty')
    plt.title('UAT Verdict: Uncertainty Interrupt Verification')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    print("✅ Visualization Generated. You should see the Uncertainty crossing the Threshold.")
else:
    print("No trajectory log found. Did the simulation run?")